In [1]:
import urllib
from bs4 import BeautifulSoup
from pymongo import MongoClient
from datetime import datetime
import calendar
#import scrapy
import re
from bson.objectid import ObjectId









In [2]:
import pymongo
def parseEventPage(month, year):
    #Download the Page Content
#     URL = 'https://en.wikipedia.org/wiki/Portal:Current_events/%s_%s' %(month,year)
    URL = 'https://en.wikipedia.org/wiki/Portal:Current_events/%s_%s' %(month,year)
    data = urllib.request.urlopen(URL).read()
    soup = BeautifulSoup(data,'html.parser')
    #soup = BeautifulSoup(data,'lxml') 
#     print(soup)
    

    #Selecting the Title
    body = soup.find('body')
    
#     print(body)

    ## define a mongodb client, create a database names WikiEvent
    client = pymongo.MongoClient("localhost", 27017)
    #db = client['datacampdb']
#     myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# mydb = myclient["mydatabase"]
    db = client['WikiEvent']
#     print(client.list_database_names())
    serverStatusResult=db.command("serverStatus")
#     print(serverStatusResult)
#     print(db)
    check=body.find_all('tr')[1]
    print(check)
    
    ## extract date for events
    datetables = body.find_all('table', attrs={'class':'plainlinks'})
#     print (datetables)
    print ('Are you here:')
    ## events on each day identified by td tag with description class attribute
    for date, dailyevent in zip(datetables, body.find_all('td', attrs={'class':'description'})):
        print ('Hi, are you there')
        datenumber = datetime.strptime(re.sub('\s*','',date.find('span', attrs={'class':'bday dtstart published updated'}).text), '%Y-%m-%d')
        ##categories identified with dt tag
        categories = dailyevent.find_all('dt')
        
        print (categories)
        print ('date: %s' %datenumber)
         #double equal loop, category and concrete events
        for category,eventcategory in zip(categories,dailyevent.find_all('ul', recursive = False)):
            categorytext = category.text
            print ('category: %s' %categorytext)
            ## single event identified with li tag. if event has specific event entity page, there will be one more li tag
            for singleevent in eventcategory.find_all('li', recursive=False):
                ## event has wikipage
                if singleevent.find('ul'):

                    singles = singleevent.find_all('ul')
                   
                    singleeventsummary = singles.pop().text  ## last li tag, event summary

                    ## event title may have multiple subtitles

                    title = [st.get('href') for st in singleevent.find_all('a', recursive=False)]

                    ## subtitle exists
                    if singles:

                        title = title + [single.find('a').get('href') for single in singles]


                    entities = [entitylink.get('href') for entitylink in singleevent.find('ul').find('li').find_all('a') if entitylink.get('title')]
                    external = singleevent.find_all('a', attrs={'class': 'external text'})
                    external_link = [e.get('href') for e in external if external]

                    print ('event title: ' )
                    print (title)
                
                else:
                    title = False
                    singleeventsummary = singleevent.text
                    entities = [entitylink.get('title') for entitylink in singleevent.find_all('a') if entitylink.get('title')]
                    
                    external = singleevent.find_all('a', attrs={'class':'external text'})
                    external_link = [e.get('href') for e in external if external]
                print ('event summary: %s' %singleeventsummary)
                myrecord={"date": datenumber,
                          "category": categorytext,
                          "event_title": title,
                          "event_summary": singleeventsummary,
                          "entities": entities,
			              "person": False,
			              "external_link": external_link
                         }
                db.events_entity_title.insert_one(myrecord)
                
#                     print(myrecord)
                #document = client.db.collection.find_one({'_id': ObjectId(myrecord)})
                 




In [3]:


import urllib.request

In [4]:
# if __name__ == '__main__':

#     #body = getEventPage('January', '2017')
#     y = '2017'
#     m = 'January'
    
#     parseEventPage(m,y)
                


In [5]:
if __name__ == '__main__':

    #body = getEventPage('January', '2017')
    year = ['2015','2016','2017','2018','2019']
    month = [calendar.month_name[i].capitalize()for i in range(1,13)]
    print(month)
    month1= [calendar.month_name[i].capitalize()for i in range(1,5)]
    #month=month.capitalize()
    
    for y in year:
            if y!='2019':
                for m in month:
                    print (y,m)
                    parseEventPage(m,y)
            elif y=='2019':
                for m in month1:
                    print (y,m)
                    parseEventPage(m,y)
                


['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
2015 January
<tr>
<td>
<table class="plainlinks" role="presentation" style="display:table; width:100%; background-color:#cef2e0; border-spacing:0; font-weight:bold;box-sizing:border-box;">
<tbody><tr>
<td style="text-align:left;font-size:11pt; padding:5px; margin-left:0px; width:100%"><span class="summary"><span style="display:none;">Current events of </span>January 1, 2015<span style="display:none"> (<span class="bday dtstart published updated">2015-01-01</span>)</span> (Thursday)</span>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="//en.wikipedia.org/w/index.php?title=Portal:Current_events/2015_January_1&amp;action=edit&amp;editintro=Portal:Current_events/Edit_instructions">edit</a></small>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink nopri

event summary: A suicide bomber kills herself and a policeman in an attack on a police station in a popular tourist district in the Turkish city of Istanbul. (New York Times)
category: Arts and culture
event summary: Officials in the American state of Massachusetts open a time capsule left behind by founding fathers Paul Revere and Samuel Adams. (Reuters via Yahoo! News)
category: Business and economics
event summary: American luxury fashion company Coach, Inc. agrees to purchase shoe brand Stuart Weitzman for $574 million. (Wall Street Journal)
event summary: The price of crude oil drops to $48 per barrel, the lowest since April of 2009. (KGO)
category: Disasters and accidents
event summary: A military truck collides with a bus carrying members of the National Guard of Ukraine in Ukraine, killing 12 soldiers and injuring 20 others. (BBC)
event summary: Two commuter trains collide at Mesquita, Rio de Janeiro in Brazil, injuring 158 people. (AFP via News24)
category: Health
event summar

Hi, are you there
[<dt>Armed conflicts and attacks</dt>, <dt>Business and economy</dt>, <dt>Disaster and accidents</dt>, <dt>Law and crime</dt>, <dt>Politics and elections</dt>]
date: 2015-01-20 00:00:00
category: Armed conflicts and attacks
event title: 
['/wiki/Shia_insurgency_in_Yemen']
event summary: Houthi rebels take over the residence of the President amidst calls by the rebel leader for negotiations to adjust the country's power structure. (The Washington Post)
event summary: The Islamic State of Iraq and the Levant threatens to kill two Japanese citizens unless it receives a ransom of $200 million. (Reuters)
event summary: Boko Haram leader Abubakar Shekau claims responsibility for the attack on the town of Baga, Nigeria in which an unknown number of civilians were killed. (Reuters via ABC News Australia)
category: Business and economy
event summary: Oil services company Baker Hughes will lay off 7,000 people due to falling crude oil prices. (AP via ABC News America)
category:

<tr>
<td>
<table class="plainlinks" role="presentation" style="display:table; width:100%; background-color:#cef2e0; border-spacing:0; font-weight:bold;box-sizing:border-box;">
<tbody><tr>
<td style="text-align:left;font-size:11pt; padding:5px; margin-left:0px; width:100%"><span class="summary"><span style="display:none;">Current events of </span>February 1, 2015<span style="display:none"> (<span class="bday dtstart published updated">2015-02-01</span>)</span> (Sunday)</span>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="//en.wikipedia.org/w/index.php?title=Portal:Current_events/2015_February_1&amp;action=edit&amp;editintro=Portal:Current_events/Edit_instructions">edit</a></small>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="//en.wikipedia.org/w/index.php?title=Portal:Current_events/2015_February_1&amp;action=history">histor

category: Politics
event summary: Premier Annastacia Palaszczuk is sworn in at Government House in Brisbane after officially winning minority government in the Queensland state election, 2015.
category: Arts and culture
event summary: Pope Francis appoints twenty new cardinals resulting in a majority of members of the College of Cardinals being from outside Europe for the first time ever. (BBC)
event summary: At the 65th Berlin International Film Festival, the film Taxi by Iranian director Jafar Panahi wins the Golden Bear. (BBC)
event summary: French film and television actor Louis Jourdan (Gigi) dies at the age of 93. (People)
category: Disasters and accidents
event summary: There is a blizzard warning in place for eastern Massachusetts in the northeastern United States. (Boston.com)
category: Health
event summary: Twelve people die of swine flu in the Indian state of Gujarat taking the death toll to 136 since January with 130 new cases reported. (IBN live)
category: International re

...

event title: 
['/wiki/LIGO', '/wiki/Virgo_interferometer']
event summary: A gravitational wave signal called GW170817 is observed by the LIGO/Virgo-collaboration. A Neutron star collision triggered the gravitational waves which "also created heavy elements such as gold, platinum and lead". (CNN)
Hi, are you there
[<dt>Armed conflicts and attacks</dt>, <dt>Arts and culture</dt>, <dt>Business and economy</dt>, <dt>Disasters and accidents</dt>, <dt>Politics and elections</dt>]
date: 2017-08-18 00:00:00
category: Armed conflicts and attacks
event title: 
['/wiki/2017_Barcelona_attacks']
event summary: Catalan police in the coastal town of Cambrils kill five suspected terrorists in a counter-terror operation to stop a second van attack. The police reported the attackers were wearing explosive vests, however they were later discovered to be fakes. (BBC)
event title: 
['/wiki/2017_Turku_stabbing']
event summary: At least two people are reportedly killed, and several others wounded in a stabbi

<tr>
<td style="text-align:left;font-size:11pt; padding:5px; margin-left:0px; width:100%"><span class="summary"><span style="display:none;">Current events of </span>September 1, 2017<span style="display:none"> (<span class="bday dtstart published updated">2017-09-01</span>)</span> (Friday)</span>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="//en.wikipedia.org/w/index.php?title=Portal:Current_events/2017_September_1&amp;action=edit&amp;editintro=Portal:Current_events/Edit_instructions">edit</a></small>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="//en.wikipedia.org/w/index.php?title=Portal:Current_events/2017_September_1&amp;action=history">history</a></small>
</td>
<td style="text-align:right; padding:5px; margin-left:0px"><small class="editlink noprint"><a class="external text" href="//en.wikipedia.org/w/index.php?title=P

category: Arts and culture
event title: 
['/wiki/Education_in_the_United_States']
event summary: Saturday's ACT U.S. college entrance exam test is cancelled at some of its international test centers because of leaks of test materials. (Reuters)
category: Business and economy
event title: 
['/wiki/Identity_theft_in_the_United_States']
event summary: More than 143 million people in the U.S. may have had their social security and driver's license numbers compromised due to a data breach on the credit rating firm Equifax. (The New York Times) (The Washington Post) (AP)
event title: 
['/wiki/American_business_history']
event summary: Amazon.com announces bidding for a second headquarters campus in North America, to house 50,000 workers and cost $5 billion USD (The New York Times)
event title: 
['/wiki/Deferred_Action_for_Childhood_Arrivals']
event summary: Fifteen states sue the Trump administration over potential violations of the Due Process Clause as a result of actions concerning the re

Hi, are you there
[<dt>Armed conflicts and attacks</dt>, <dt>Business and economy</dt>, <dt>Law and crime</dt>, <dt>Politics and elections</dt>, <dt>Science and technology</dt>]
date: 2017-09-26 00:00:00
category: Armed conflicts and attacks
event title: 
['/wiki/2017_Har_Adar_shooting']
event summary: Three Israelis are killed and one other is wounded after a Palestinian gunman opens fire in an entrance to the Israeli settlement of Har Adar in the West Bank. (Reuters)
category: Business and economy
event title: 
['/wiki/Mass_surveillance_in_the_United_States']
event summary: The United States Department of Homeland Security plans to collect the social media information of all immigrants, including green card holders and naturalized citizens, beginning on October 18, 2017. (CNET)
event title: 
['/wiki/Social_networking_service']
event summary: Twitter is experimenting with doubling its 140 character limit to 280 characters for all languages except Japanese, Chinese and Korean.  (USA To

<tr><th style="text-align:center;width:14%">S</th><th style="text-align:center;width:14%">M</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">W</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">F</th><th style="text-align:center;width:14%">S</th></tr>
Are you here:
2017 November
<tr><th style="text-align:center;width:14%">S</th><th style="text-align:center;width:14%">M</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">W</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">F</th><th style="text-align:center;width:14%">S</th></tr>
Are you here:
2017 December
<tr><th style="text-align:center;width:14%">S</th><th style="text-align:center;width:14%">M</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">W</th><th style="text-align:center;width:14%">T</th><th style="text-align:center;width:14%">F</th>